# Tensorflow to Qkeras to Hls4ml


In [1]:
import numpy as np

seed = 0
np.random.seed(seed)

import tensorflow as tf

tf.random.set_seed(seed)


import sys

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import dataset
import plotting
import qkeras
import keras

2024-02-07 21:13:19.885107: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 21:13:19.907874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 21:13:19.907894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 21:13:19.908550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 21:13:19.912955: I tensorflow/core/platform/cpu_feature_guar

# fetch dataset


In [2]:
BATCH_SIZE = 64
train_images_rgx = "../dataset/train_set_4x/*/*.jpeg"
valid_images_rgx = "../dataset/test_set_4x/*/*.jpeg"

train_ds, val_ds, nclasses = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE, 128
)

2024-02-07 21:13:21.187270: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-07 21:13:21.210067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-07 21:13:21.210199: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# load the model


In [3]:
from qkeras.utils import load_qmodel
from qkeras.estimate import print_qstats

model_path = "/home/webphy/Desktop/dnn_processor/hls4ml/saved_qmodels/handmade_model.h5"

qmodel = load_qmodel(
    model_path,
    compile=False,
)
qmodel.compile(
    "adam", loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)

score = qmodel.evaluate(val_ds)
print("Accuracy: {}".format(score[1]))

print_qstats(qmodel)

2024-02-07 21:13:25.327375: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


5/5 [==============================] - 2s 33ms/step - loss: 0.4842 - accuracy: 0.8238
Accuracy: 0.8237547874450684
Instructions for updating:
Use ref() instead.


/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(
Instructions for updating:
Use ref() instead.



Number of operations in model:
    q_conv2d                      : 196608 (smult_8_8)
    q_conv2d_1                    : 129600 (smult_8_8)
    q_conv2d_2                    : 18000 (smult_8_8)
    q_dense                       : 5625  (smult_8_4)

Number of operation types in model:
    smult_8_4                     : 5625
    smult_8_8                     : 344208

Weight profiling:
    q_conv2d_weights               : 192   (8-bit unit)
    q_conv2d_bias                  : 4     (8-bit unit)
    q_conv2d_1_weights             : 576   (8-bit unit)
    q_conv2d_1_bias                : 16    (8-bit unit)
    q_conv2d_2_weights             : 80    (8-bit unit)
    q_conv2d_2_bias                : 5     (8-bit unit)
    q_dense_weights                : 5625  (8-bit unit)
    q_dense_bias                   : 5     (8-bit unit)

Weight sparsity:
... quantizing model
    q_conv2d                       : 0.0102
    q_conv2d_1                     : 0.0203
    q_conv2d_2                     

# extract one layer


In [4]:
x = x_in = keras.layers.Input(shape=(128,128,3))

x = qmodel.layers[1](x)
x = qmodel.layers[2](x)
x = qmodel.layers[3](x)

qmodel.summary()

one_layer_model = keras.models.Model(x_in, x)
one_layer_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 q_activation (QActivation)  (None, 128, 128, 3)       0         
                                                                 
 q_conv2d (QConv2D)          (None, 32, 32, 4)         196       
                                                                 
 q_activation_1 (QActivatio  (None, 32, 32, 4)         0         
 n)                                                              
                                                                 
 q_conv2d_1 (QConv2D)        (None, 15, 15, 16)        592       
                                                                 
 q_activation_2 (QActivatio  (None, 15, 15, 16)        0         
 n)                                                          

In [5]:
model = keras.models.load_model(
    "/home/webphy/Desktop/dnn_processor/neural_network/handmade/saved_models/manual_test_4x/model.h5"
)
model.evaluate(val_ds)

x = x_in = keras.layers.Input(shape=(32, 32, 4))
for l in model.layers[3:]:
    print(l.name)
    x = l(x)

last_layer_model = keras.models.Model(x_in, x)
last_layer_model.summary()

5/5 [==============================] - 0s 21ms/step - loss: 0.3979 - accuracy: 0.8697
conv2d_4
activation_3
conv2d_5
flatten_1
dense_1
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 4)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 15, 15, 16)        592       
                                                                 
 activation_3 (Activation)   (None, 15, 15, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 15, 15, 5)         85        
                                                                 
 flatten_1 (Flatten)         (None, 1125)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 5

# convert quantized model to HLS4ml


In [6]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(
    one_layer_model,
    granularity="name",
    default_precision="fixed<32,16>",
    default_reuse_factor=9000,
)

config["Model"]["Strategy"] = "Resource"

for layer in config["LayerName"].keys():
    config["LayerName"][layer]["Strategy"] = "Resource"

#config["LayerName"]["q_dense"]["Strategy"] = "Resource"
#config["LayerName"]["q_dense"]["Precision"] = "fixed<4,1>"


Interpreting Model
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 128, 128, 3]], output shape: [None, 128, 128, 3]
Layer name: q_activation, layer type: Activation, input shapes: [[None, 128, 128, 3]], output shape: [None, 128, 128, 3]
Layer name: q_conv2d, layer type: QConv2D, input shapes: [[None, 128, 128, 3]], output shape: [None, 32, 32, 4]
Layer name: q_activation_1, layer type: Activation, input shapes: [[None, 32, 32, 4]], output shape: [None, 32, 32, 4]


/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)
/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [7]:
plotting.print_dict(config)

Model
  Precision:         fixed<32,16>
  ReuseFactor:       9000
  Strategy:          Resource
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  input_2
    Trace:           False
    Precision
      result:        fixed<32,16>
    Strategy:        Resource
  q_activation
    Trace:           False
    Precision
      result:        ufixed<8,0,RND_CONV,SAT>
    Strategy:        Resource
  q_conv2d
    Trace:           False
    Precision
      result:        fixed<32,16>
      weight:        fixed<8,1>
      bias:          fixed<8,1>
    Strategy:        Resource
  q_conv2d_linear
    Trace:           False
    Precision
      result:        fixed<32,16>
    Strategy:        Resource
  q_activation_1
    Trace:           False
    Precision
      result:        ufixed<8,0,RND_CONV,SAT>
    Strategy:        Resource


# build for pynq without DMA

In [ ]:

hls_model = hls4ml.converters.convert_from_keras_model(
    one_layer_model,
    hls_config=config,
    output_dir=f"one_layer_model/hls4ml_prj",
    part="xc7z020-clg400-1",
    io_type="io_stream",
)

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
hls_model.compile()

In [ ]:
classes = ["acf_alv", "acf_chap", "reboco", "tijolo_alv", "tijolo_chap"]
X_test = np.load("../dataset/x_test_set_4x.npy")
y_test = np.load("../dataset/y_test_set_4x.npy")

y_hls = hls_model.predict(np.ascontiguousarray(X_test))
y_hls = y_hls.reshape((-1, 32, 32, 4))
y_last_layers = last_layer_model.predict(y_hls)

np.save("one_layer_model/y_hls.npy", y_hls)
np.save("one_layer_model/y_last_layers.npy", y_last_layers)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


print(
    "Accuracy hls4ml: {}".format(
        accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_last_layers, axis=1))
    )
)

fig, ax = plt.subplots(figsize=(9, 9))
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(y_test, y_last_layers, classes, linestyle=":")

from matplotlib.lines import Line2D

lines = [Line2D([0], [0], ls="-")]
from matplotlib.legend import Legend

leg = Legend(
    ax,
    lines,
    labels=["hls4ml"],
    loc="lower right",
    frameon=False,
)
ax.add_artist(leg)

In [ ]:
hls_model.build(csim=False)

In [ ]:
hls4ml.report.read_vivado_report(f"one_layer_model/hls4ml_prj")

# build for Pynq

In [8]:
output_dir=f"deploy/one_layer_model/hls4ml_prj"

hls_model = hls4ml.converters.convert_from_keras_model(
    one_layer_model,
    hls_config=config,
    output_dir=output_dir,
    part="xc7z020-clg400-1",
    io_type="io_stream",
    backend='VivadoAccelerator',
    board='pynq-z2'
)

hls_model.compile()
hls_model.build(csim=False, export=True, bitfile=True)

Interpreting Model
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 128, 128, 3]], output shape: [None, 128, 128, 3]
Layer name: q_activation, layer type: Activation, input shapes: [[None, 128, 128, 3]], output shape: [None, 128, 128, 3]
Layer name: q_conv2d, layer type: QConv2D, input shapes: [[None, 128, 128, 3]], output shape: [None, 32, 32, 4]
Layer name: q_activation_1, layer type: Activation, input shapes: [[None, 32, 32, 4]], output shape: [None, 32, 32, 4]
Creating HLS model
Writing HLS project


/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2020.1 (64-bit)
  **** SW Build 2902540 on Wed May 27 19:54:35 MDT 2020
  **** IP Build 2902112 on Wed May 27 22:43:36 MDT 2020
    ** Copyright 1986-2020 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2020.1/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2020.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'webphy' on host 'wesley-XPS-8950' (Linux_x86_64 version 6.5.0-17-generic) on Wed Feb 07 21:13:52 -03 2024
INFO: [HLS 200-10] On os Ubuntu 22.04.3 LTS
INFO: [HLS 200-10] In directory '/home/webphy/Desktop/dnn_processor/hls4ml/deploy/one_layer_model/hls4ml_prj'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/webphy/Desktop/dnn_processor/hls4ml/deploy/one_layer_model/hls4ml_prj/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject_axi.cpp' to the project
INFO: [HLS 200-10] Adding des

{'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '6.279',
  'BestLatency': '3292996',
  'WorstLatency': '3293190',
  'IntervalMin': '475138',
  'IntervalMax': '3293186',
  'BRAM_18K': '133',
  'DSP': '0',
  'FF': '6594',
  'LUT': '12135',
  'URAM': '0',
  'AvailableBRAM_18K': '280',
  'AvailableDSP': '220',
  'AvailableFF': '106400',
  'AvailableLUT': '53200',
  'AvailableURAM': '0'},
 'TimingReport': {'WNS': 0.969,
  'TNS': 0.0,
  'WHS': 0.021,
  'THS': 0.0,
  'WPWS': 3.75,
  'TPWS': 0.0}}

In [9]:
hls4ml.report.read_vivado_report(output_dir)

Found 1 solution(s) in deploy/one_layer_model/hls4ml_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject_axi'
* Date:           Wed Feb  7 21:14:10 2024

* Version:        2020.1 (Build 2897737 on Wed May 27 20:21:37 MDT 2020)
* Project:        myproject_prj
* Solution:       solution1
* Product family: zynq
* Target device:  xc7z020-clg400-1


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 6.279 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+-----------+-----------+--------+---------+----------+
    |  Latency (cycles) |   Latency (absolute)  |     Interval     | Pipeline |
    |   min   |   max   |    min    |    max    |   min  |   max   |   Type   |


# FIFO depth optimization

In [ ]:
import hls4ml


config["Flows"] = ["vivado:fifo_depth_optimization"]
hls4ml.model.optimizer.get_optimizer("vivado:fifo_depth_optimization").configure(
    profiling_fifo_depth=100_000
)


hls_model = hls4ml.converters.convert_from_keras_model(
    qkeras_model,
    hls_config=config,
    io_type="io_stream",
    output_dir=f"deploy/qmodel_{trial_id}_fifo_opt/hls4ml_prj",
    part="xc7z020clg400-1",
    backend="Vivado",
)

hls_model.build(reset=False, csim=True, synth=True, cosim=True)

In [ ]:
hls4ml.report.read_vivado_report(f"deploy/qmodel_{trial_id}_fifo_opt/hls4ml_prj")

# FIFO depth optimization for PYNQ

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    qkeras_model,
    hls_config=config,
    output_dir=f"deploy/qmodel_{trial_id}_fifo_opt_pynq/hls4ml_prj",
    part="xc7z020clg400-1",
    io_type="io_stream",
    backend='VivadoAccelerator',
    board='pynq-z2'
)

In [ ]:
hls_model.compile()

In [ ]:
hls_model.build(
    reset=False, csim=True, synth=True, cosim=True, export=True, bitfile=True
)

In [ ]:
hls4ml.report.read_vivado_report(f"deploy/qmodel_{trial_id}_fifo_opt_pynq/hls4ml_prj")